In [1]:
# %pip install --quiet --no-cache --force git+https://github.com/romnn/lsde2021

In [2]:
%load_ext autoreload
%autoreload 2
import pyspark
import lsde2021.download as dl
import lsde2021.aggregate as agg
from lsde2021.types import PathLike
from pyspark.sql import SparkSession
from functools import partial
from pathlib import Path
import datetime

In [3]:
MAX_MEMORY = "60G"

spark = SparkSession \
    .builder \
    .appName("EDA") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .getOrCreate()
sc = spark.sparkContext

21/10/03 23:43:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
for year in [2018, 2019, 2020, 2021]:
    hour_range = list(dl.datetime_range(
        datetime.datetime(year, 1, 1, hour=0),
        datetime.datetime(year, 1, 8, hour=0)
    ))
    assert len(hour_range) == 7 * 24 + 1
    
    dest = Path("./wikimedia_data")
    downloaded = sc.parallelize(dl.wikimedia_files(hour_range)) \
        .map(partial(dl.download_handler, dest=dest, force=False)) \
        .collect()

    downloaded_files = [p for p in (dest / f"{year}/{year}-01").glob("**/*.gz")]
    print(downloaded_files[:10])

downloading wikimedia_data/2018/2018-01/pageviews-20180103-120000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180106-000000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180105-040000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180104-080000.gzdownloading wikimedia_data/2018/2018-01/pageviews-20180105-140000.gz

downloading wikimedia_data/2018/2018-01/pageviews-20180101-100000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180104-180000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180103-020000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180101-200000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180103-220000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180102-160000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180107-060000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180101-000000.gz
downloading wikimedia_data/2018/2018-01/pageviews-20180106-200000.gz
downloading wikimedia_data/2018/20

using existing file wikimedia_data/2018/2018-01/pageviews-20180103-140000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180103-150000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180102-180000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180102-190000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180103-060000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180103-070000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180104-010000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180104-020000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180106-040000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180106-050000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180101-040000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180101-050000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180106-230000.gz ...
downloading wikimed

using existing file wikimedia_data/2018/2018-01/pageviews-20180103-100000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180103-110000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180105-120000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180105-130000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180102-210000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180102-220000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180104-140000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180104-150000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180104-060000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180104-070000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180105-000000.gz ...
downloading wikimedia_data/2018/2018-01/pageviews-20180105-010000.gz
using existing file wikimedia_data/2018/2018-01/pageviews-20180102-130000.gz ...
downloading wikimed

[PosixPath('wikimedia_data/2018/2018-01/pageviews-20180101-110000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180104-030000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180104-160000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180104-020000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180104-190000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180102-160000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180103-150000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180106-070000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180104-090000.gz'), PosixPath('wikimedia_data/2018/2018-01/pageviews-20180102-150000.gz')]


downloading wikimedia_data/2019/2019-01/pageviews-20190103-020000.gz
using existing file wikimedia_data/2019/2019-01/pageviews-20190107-060000.gz ...
downloading wikimedia_data/2019/2019-01/pageviews-20190107-070000.gz
using existing file wikimedia_data/2019/2019-01/pageviews-20190101-100000.gz ...
downloading wikimedia_data/2019/2019-01/pageviews-20190101-110000.gz
using existing file wikimedia_data/2019/2019-01/pageviews-20190102-060000.gz ...
downloading wikimedia_data/2019/2019-01/pageviews-20190102-070000.gz
using existing file wikimedia_data/2019/2019-01/pageviews-20190101-000000.gz ...
downloading wikimedia_data/2019/2019-01/pageviews-20190101-010000.gz
using existing file wikimedia_data/2019/2019-01/pageviews-20190103-020000.gz ...
downloading wikimedia_data/2019/2019-01/pageviews-20190103-030000.gz
using existing file wikimedia_data/2019/2019-01/pageviews-20190104-080000.gz ...
downloading wikimedia_data/2019/2019-01/pageviews-20190104-090000.gz
using existing file wikimedia_d

...

using existing file wikimedia_data/2021/2021-01/pageviews-20210102-060000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210102-070000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210105-040000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210105-050000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210103-020000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210103-030000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210101-100000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210101-110000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210101-000000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210101-010000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210107-060000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210107-070000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210106-000000.gz ...
downloading wikimed

using existing file wikimedia_data/2021/2021-01/pageviews-20210106-040000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210106-050000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210106-230000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210107-000000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210102-180000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210102-190000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210101-230000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210102-000000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210104-200000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210104-210000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210103-070000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210103-080000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210104-010000.gz ...
downloading wikimed

using existing file wikimedia_data/2021/2021-01/pageviews-20210104-050000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210104-060000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210104-230000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210105-000000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210104-140000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210104-150000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210107-040000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210107-050000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210106-090000.gz ...
using existing file wikimedia_data/2021/2021-01/pageviews-20210107-130000.gz ...
downloading wikimedia_data/2021/2021-01/pageviews-20210107-140000.gz
using existing file wikimedia_data/2021/2021-01/pageviews-20210103-110000.gz ...
using existing file wikimedia_data/2021/2021-01/pageviews-20210106-160000.gz ...
downloa

[PosixPath('wikimedia_data/2021/2021-01/pageviews-20210105-070000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210106-180000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210105-040000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210105-020000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210105-210000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210105-100000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210104-030000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210104-050000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210101-010000.gz'), PosixPath('wikimedia_data/2021/2021-01/pageviews-20210104-180000.gz')]


using existing file wikimedia_data/2021/2021-01/pageviews-20210108-000000.gz ...


In [8]:
for year in [2018, 2019, 2020, 2021]:
    daily_date_range = list(dl.datetime_range(
        datetime.datetime(year, 1, 1, hour=0),
        datetime.datetime(year, 1, 1, hour=0),
        interval=datetime.timedelta(days=1)
    ))
    print(len(daily_date_range))
    assert len(daily_date_range) == 1
    
    # aggregate days here and store to parquet
    for date in daily_date_range:
        agg.aggregate_daily_pageviews(date.date(), spark=spark, src=dest, dest=dest / "daily")

1


wrote wikimedia_data/daily/2018/2018-1-1.parquet
1


21/10/04 10:03:25 ERROR Executor: Exception in task 90.0 in stage 794.0 (TID 15360)
java.io.FileNotFoundException: /tmp/blockmgr-134be020-7293-4b02-9d7d-941ad3049d19/18/shuffle_593_15360_0.index.77040512-b033-4c6c-8dcd-f3e26b9a336f (No space left on device)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:298)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:237)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:187)
	at org.apache.spark.shuffle.IndexShuffleBlockResolver.writeIndexFileAndCommit(IndexShuffleBlockResolver.scala:295)
	at org.apache.spark.shuffle.sort.io.LocalDiskShuffleMapOutputWriter.commitAllPartitions(LocalDiskShuffleMapOutputWriter.java:118)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.writePartitionedData(BypassMergeSortShuffleWriter.java:222)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWrit

failed to save daily data wikimedia_data/daily/2019/2019-1-1.parquet: An error occurred while calling o22608.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.Sp

failed to save daily data wikimedia_data/daily/2020/2020-1-1.parquet: [Errno 28] No space left on device: 'wikimedia_data/daily/2020'
Traceback (most recent call last):
  File "/home/jovyan/work/lsde2021/aggregate.py", line 81, in aggregate_daily_pageviews
    daily_out.parent.mkdir(parents=True, exist_ok=True)
  File "/opt/conda/lib/python3.9/pathlib.py", line 1313, in mkdir
    self._accessor.mkdir(self, mode)
OSError: [Errno 28] No space left on device: 'wikimedia_data/daily/2020'

1
failed to save daily data wikimedia_data/daily/2021/2021-1-1.parquet: [Errno 28] No space left on device: 'wikimedia_data/daily/2021'
Traceback (most recent call last):
  File "/home/jovyan/work/lsde2021/aggregate.py", line 81, in aggregate_daily_pageviews
    daily_out.parent.mkdir(parents=True, exist_ok=True)
  File "/opt/conda/lib/python3.9/pathlib.py", line 1313, in mkdir
    self._accessor.mkdir(self, mode)
OSError: [Errno 28] No space left on device: 'wikimedia_data/daily/2021'



In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType
import pyspark.sql.functions as F
import traceback

# see https://stackoverflow.com/questions/51217168/wikipedia-pageviews-analysis
# domain_code
# page_title
# count_views
# total_response_size (no longer maintained)

def aggregate_daily_pageviews(date: datetime.date, src: PathLike, dest: PathLike) -> PathLike:
    schema = StructType([
        StructField("domain_code", StringType(), True),
        StructField("page_title", StringType(), True),
        StructField("view_count", LongType(), True),
        StructField("total_response_size", IntegerType(), True)
    ])

    csv_loader = spark.read.format("csv").option("sep", ' ')
    
    daily = None
    daily_out = dest / Path("/".join(dl.wikimedia_daily_local_file(date)))
    
    for hour in range(24):
        current = datetime.datetime.combine(date, datetime.time.min) + datetime.timedelta(hours=hour)
        file = src / Path("/".join(dl.wikimedia_local_file(current)))
        # print(file)
        # continue
        try:
            df = csv_loader.load(str(file), schema=schema)
            if daily is None:
                daily = df
            else:
                daily = df \
                    .select("domain_code", "page_title", F.col("view_count").alias("view_count2")) \
                    .join(daily, on=["domain_code", "page_title"], how="outer") \
                    .fillna(value=0)
                daily = daily \
                    .withColumn('view_count_sum', sum([daily["view_count"], daily["view_count2"]])) \
                    .select("domain_code", "page_title", F.col("view_count_sum").alias("view_count"))
        except Exception as e:
            print(f"failed to load {file}: {e}")
            print(traceback.format_exc())
    
    if daily:
        try:
            daily = daily \
                .sort(F.col("view_count").desc()) \
                .repartition(F.col("domain_code"))
            daily.show()
            daily_out.parent.mkdir(parents=True, exist_ok=True)
            daily.write.format("parquet").partitionBy("domain_code").mode("overwrite").save(str(daily_out))
            print(f"wrote {daily_out}")
            # print(date, daily.count())
        except Exception as e:
            print(f"failed to save daily data {daily_out}: {e}")
            print(traceback.format_exc())
    return daily_out
    
for date in daily_date_range:
    agg.aggregate_daily_pageviews(spark, date.date(), src=dest, dest=dest / "daily")

In [ ]:
%reload_ext autoreload
%autoreload 2
import lsde2021.aggregate as agg
hourly = sc.parallelize([d.date() for d in daily_date_range][:1]) \
    .map(partial(agg.aggregate_daily_pageviews, src=dest, dest=dest / "daily")) \
    .collect()
print(len(hourly))
print(hourly[:10])

In [ ]:
# sc.stop()

In [ ]:
csv_loader = spark.read.format("csv") \
            .option("sep", ' ')
           #.option("header", "true") \
           #.option("delimiter", "|") \
           #.option("inferschema", "true")
for date, file in downloaded:
    df = csv_loader.load(str(file))
    df.show(1)
    break

In [ ]:
schema = StructType([
    StructField("domain", StringType(), True),
    StructField("pagename", StringType(), True),
    StructField("count", StringType(), True),
   StructField("responsebytes", StringType(), True)
])


df = spark.read.option("sep"," ").csv("/mnt/group29/test.gz", schema=schema)
df.show(1)